## Programmation concurente - <span style="color:blue;">CORRECTION</span>

**Sources** : Numérique et Sciences Informatiques - <span class="codepy">ellipses</span>

**Ressources** : <a href="https://webge.fr/dokuwiki/doku.php?id=python:accueilpython" target="_blank"><input type="button" value="Wiki Python sur WebGE"></a> 

### Sommaire
<ol>
    <li>Introduction
        <ol>
            <li>Les processus</li>
            <li>Les threads</li>
            <li>Quelle est la différence entre Thread et Processus ?</li>
        </ol>
    </li>
    <li>Premier exemple - Comptage en parallèle</li>
    <li>Deuxième exemple - Compteur global partagé</li>
        <ol>
            <li>Problème de concurrence</li>
            <li>Correction du problème de concurence</li>
        </ol>
    <li>Troisième exemple - Interblocage</li>
    <li>Synthèse</li>
    <li>Résumé</li>
</ol>

### 1. Introduction
#### 1.a Les processus
<em>Grâce à leur système d'exploitation **multitâche**, les ordinateurs exécutent plusieurs programmes de façon **concurrente**. L'exécution d'un programme s'appelle un **processus**. C'est le système d'exploitation, et en particulier l'**ordonnanceur** (une des fonctionnalités du **noyau**), qui détermine quel processus s'exécute à un instant donné. Le fait pour un processus d'être interrompu s'appelle une **commutation de contexte**. Plusieurs processus s'exécutant de façon concurrente peuvent s'**interbloquer** s'ils attendent de pouvoir accéder à un même ensemble de **ressources en accès exclusif**. </em><br>
<img src="img/se.png"> <br>
#### 1.b Les threads
<em>Les **threads** ou processus légers sont des "sous-processus", démarrés par un processus et s'exécutant de manière concurrente avec le reste du programme. L'accès à des ressources par plusieurs threads peut être protégé par des **verrous**. Une portion de code comprise entre l'acquisition et le relâchement d'un verrou s'appelle une **section critique**. <br>
Le module threading de la bibliothèque standard Python permet de démarrer des threads.</em><br>
#### 1.c Quelle est la différence entre Thread et Processus ?
<em>Les threads (du même processus) s'exécutent dans un espace mémoire partagé, tandis que les processus s'exécutent dans des espaces mémoire différents.</em>
<img src="img/threads.png"> 

### Objectifs
S'initier à la **programmation multithread** en Python et **illustrer les problèmes de concurrence et d'interblocage** dans un cadre plus contrôlé que dans un système d'exploitation.

### 2. Premier exemple - Comptage en parallèle
<em>Dans le code ci-dessous, le programme principal effectue une boucle et appelle quatre fois l'expression <span style="font-family:Consolas;font-weight:bold;font-style:normal">threading.Thread(target=hello, arg=[n])</span>. Cette dernière crée un objet de type **Thread**. La variable <span class="code"><strong>t</strong></span> contient l'objet **Thread** créé. La méthode **start()** lance l'exécution de la fonction en tâche de fond. Cette méthode rend directement la main et le programme principal continue de s'exécuter de façon concurrente au thread démarré. Une fois la boucle exécutée, ce programme comporte cinq *threads* : les quatre démarrés par start() et le programme principal.</em>

In [ ]:
# Programmation concurente - Comptage 
import threading

def hello(num):
    for i in range(5):
        print(f"je suis le thread {num} et ma valeur est {i}")
    print(f"----- Fin du thread {num} -----")


for numth in range(4):
    th = threading.Thread(target=hello, args=[numth]) # l'argument de type target est une fonction et l'argument arg un tableau
    th.start()                                        # d'arguments passés à la fonction

> **Activité 1 <br>
> Exécutez** plusieurs fois le code ci-dessus. Que remarquez-vous ?

<p style="color:blue; font-weight:bold">CORRECTION 1</p>
<ul style="color:blue;">
    <li>Les threads alternent leur exécution au gré des commutations de contexte.</li>
    <li>Deux exécutions successives donnent des affichages différents.</li>
</ul>

<table><tr><td style="color:red; font-size:14px"><strong>REMARQUE</strong> : L'ordre dans lequel sont démarrés les threads ne donne aucune indication sur l'ordre dans lequel ils peuvent se terminer.</td></tr></table>

> **Activité 2. En vous inspirant du programme ci-dessus, écrivez** un programme qui exécute **quatre** fonctions concurrentes affichant plusieurs fois un message de bienvenue personnalisé. Une fonction produira un maximum de **dix** fois le message.<br>

> *Exemple de résultats attendus*<br>
Bonjour, je suis le thread 0 et ceci est le message 1<br>
... <br>
Message de bienvenue du thread 1 qui transmet son message 3<br>
... <br>


In [ ]:
# Correction 2
# Programmation concurente - Messages différents
# Correction

import threading
'''
num : numéro du thread
msg : message affiché
nb : nombre de messages
'''
def hello(num,msg,nb):
    for i in range(nb):
        print(f"{msg} {i}")
    print(f"----- Fin du thread {num} -----")
    
nb=[10,7,5,8]
msg=["Bonjour, je suis le thread 0 et ceci est le message ","Message de bienvenue du thread 1 qui transmet le message ",
     "Salut, le thread 2 vous envoie le message ", "Hé, le thread 3 aussi envoie son message "]

for num in range(len(nb)):
    t = threading.Thread(target=hello, args=[num,msg[num], nb[num]]) 
    t.start()

### 3. Deuxième exemple - Compteur global partagé
#### 3.A Problème de concurence
<em>Les threads peuvent servir à illustrer les problèmes de concurrence. Le programme ci-dessous définit une variable globale **COMPTEUR**. Comme hello() dans le programme précédent, la fonction **incrc()** ci-dessous s'exécute dans des Threads. Le programme principal déclare un tableau vide <strong>th</strong>. Il démarre ensuite quatre threads et stocke les objets correspondants dans le tableau th, après les avoir démarrés. Pour chacun des threads stockés, la méthode **join()** est appelée. Cette méthode permet d'attendre que le thread auquel on l'applique soit terminé. Si le thread est déjà terminé, la méthode se termine immédiatement.</em>


In [ ]:
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence v1
import threading

COMPTEUR = 0

def incrc():
    global COMPTEUR
    for _ in range(100000):
        COMPTEUR+=1
        #v = COMPTEUR      # Ces deux lignes pourraient être remplacées par COMPTEUR +=1. 
        #COMPTEUR = v + 1  # Elles sont utilisée ici pour simplifier les explications

th=[] # tableau de threads
for n in range(4):
    t = threading.Thread(target=incrc, args=[])
    t.start()
    th.append(t)

for t in th:
    t.join()

print(f"Valeur finale = {COMPTEUR}") # Cette ligne est exécutée lorsque tous les threads sont terminés

> **Activité 3a. Analyse** du programme <br>
> Que fait la fonction incrc ? Quelle devrait être la valeur de COMPTEUR à la fin du programme ? 

<p style="color:blue; font-weight:bold">CORRECTION 3a</p>
<ul style="color:blue;">
    <li>La fonction incrc() exécute 100000 itérations d'une boucle qui incrémente la variable COMPTEUR.</li>
    <li>A la fin du programme, la variable COMPTEUR devrait être égale à 400000.</li>
</ul>

> **Activité 3b. Exécutez** plusieurs fois le code ci-dessus. Que remarquez-vous ?

<p style="color:blue; font-weight:bold">CORRECTION 3b</p>
<p style="color:blue;">La valeur finale de COMPTEUR est égale à 400000</p>

<table><tr><td style="color:red; font-size:14px"><strong>REMARQUE</strong> : Contrairement à ce que l'on peut observer sur quelques essais, le programme ci-dessus ne fonctionne pas correctement dans la durée.</td></tr></table>

> **Activité 4** <br> 
> **Modifiez** le programme ci-dessous pour qu'il affiche les résultats lorsqu'ils sont différents de 400000. <br>

> *Exemple de résultat attendu*<br>
En attente d'une erreur<br>
Valeur finale de COMPTEUR erronée = 300000 après 2643 tests<br>
Valeur finale de COMPTEUR erronée = 300000 après 10504 tests<br>
Valeur finale de COMPTEUR erronée = 393134 après 13748 tests<br>

In [ ]:
# Correction 4
# Programmation concurente - Compteur partagé
# Illustration du problème de concurrence v2
# Correction
import threading
COMPTEUR=0
i=1

def incrc():
    global COMPTEUR
    for c in range(100000):
        COMPTEUR += 1


print("En attente d'une erreur !")

while True:
    th=[]
    for n in range(4):
        t = threading.Thread(target=incrc, args=[])
        t.start()
        th.append(t)
    for t in th:
        t.join()
    i+=1
    if COMPTEUR != 400000:
        print(f"Valeur finale de COMPTEUR = {COMPTEUR} après {i} tests") 
    COMPTEUR=0

#### 3.B Correction du problème de concurrence dans le programme "Compteur partagé"
<em>Pour corriger ce problème, il faut garantir l'<strong>accès EXCLUSIF</strong> à la variable <strong>COMPTEUR</strong> entre sa lecture et son écriture. On peut pour cela utiliser un verrou. Un <strong>verrou</strong> est un objet que l'on essaye d'acquérir. Si on est le premier à faire cette demande, on acquiert le verrou. On peut le rendre à tout moment. Si en revanche quelqu'un d'autre tient le verrou alors on est bloqué jusqu'à ce qu'il soit libéré. Des verrous munis de ces deux opérations sont disponibles dans le <strong>module threading</strong> avec le constructeur <strong>Lock()</strong>. Une fois le verrou construit, on peut tenter de l'acquérir avec la méthode <strong>acquire()</strong> et on peut le rendre avec la méthode <strong>release()</strong>.</em>

<table><tr><td style="color:blue; font-size:14px"><strong>NOTE</strong> : Une portion de code protégée par un verrou s'appelle une <strong>SECTION CRITIQUE</strong>.</td></tr></table>

> **Activité 5**<br>
>**Exécutez** le programme ci-dessous. Laissez le programme atteindre 5000 simulations avant de répondre aux questions ci-dessous. En attendant, passez à l'activité suivante.

> _Après 5000 simulations_ : <br>
a) Que remarquez-vous ?<br>
b) Expliquez pourquoi on a corrigé le problème de concurrence entre les threads t1, t2, t3 et t4.

<p style="color:blue; font-weight:bold">CORRECTION 5</p>
<ul style="color:blue;">
    <li>a) COMPTEUR = 40000, le résultat est celui attendu.</li>
    <li>b) Un thread ne peut pas incrémenter le compteur s'il ne dispose pas du verrou. Un seul thread peut disposer du verrou. Pour qu'un thread se termine, il doit avoir incrémenté le compteur 10000 fois. </li>
</ul>

In [ ]:
# Programmation concurente - Compteur partagé
# Correction du problème de concurrence avec un verrou

import threading

COMPTEUR = 0
verrou = threading.Lock() # construction du verrou
i=1

def incrc():
    global COMPTEUR
    for c in range(100000):
        verrou.acquire() # Acquisition du verrou
        v = COMPTEUR
        COMPTEUR = v + 1
        verrou.release() # Relâchement du verrou
 
while True:
    th=[]
    for n in range(4):
        t = threading.Thread(target=incrc, args=[])
        t.start()
        th.append(t)
    for t in th:
        t.join()
    i+=1
    print(f"Simulation{i}, COMPTEUR = {COMPTEUR}") 
    COMPTEUR=0

### 4. Troisième exemple - Interblocage
L'interblocage se produit lorsque des processus concurrents **s'attendent mutuellement**. L'utilisation de plusieurs verrous rend les **interblocages** possibles.
<img src="img/interblocage.png">

> **Activité 6a. Analyse** du programme ci-dessous <br>
Quel pourrait être le texte affiché par le programme : <br>
a) S'il ne se bloquait pas ? <br>
b) S'il se bloquait ? 

<p style="color:blue; font-weight:bold">CORRECTION 6a</p>
<ul style="color:blue;">a) Pas d'interblocage
    <li>f1 a acquit le verrou 1</li>   
    <li>f1 a acquit le verrou 2</li> 
    <li>f1 a relâché le verrou 2</li> 
    <li>f1 a relâché le verrou 1</li> 
    <li>f2 a acquit le verrou 2</li> 
    <li>f2 a acquit le verrou 1</li>
    <li>f2 a relâché le verrou 1</li>
    <li>f2 a relâché le verrou 2</li>
</ul>
<ul style="color:blue;">b) Interblocage
    <li>f1 a acquit le verrou 1</li>   
    <li>f2 a acquit le verrou 2</li> 
</ul>

In [ ]:
# Interblocage
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def f1():
    verrou1.acquire()
    print("f1 a acquit le verrou 1")
    i=0; j=0
    for _ in range(100):
        i+=1
        for _ in range (10000):
          j+=1  
    verrou2.acquire()
    print("f1 a acquit le verrou 2")
    verrou2.release()
    print("f1 a relâché le verrou 2")
    verrou1.release()
    print("f1 a relâché le verrou 1")
    
    
def f2():
    verrou2.acquire()
    print("f2 a acquit le verrou 2")
    verrou1.acquire()
    print("f2 a acquit le verrou 1")
    verrou1.release()
    print("f2 a relâché le verrou 1")
    verrou2.release()
    print("f2 a relâché le verrou 2")

t1 = threading.Thread(target=f1, args=[])
t2 = threading.Thread(target=f2, args=[])
t1.start()
t2.start()

> **Activité 6b** <br>
>**Diminuez** la durée des boucles pour que le programme ne se bloque plus. <br>
> a) Pour quelle durée n'a-t-on plus de blocage ? <br>
> b) Peut-on conserver cette solution ? Pourquoi ?

<p style="color:blue; font-weight:bold">CORRECTION 6b</p>
<ul style="color:blue;">
    <li>Exemple : le blocage ne se produit plus pour 10000 suivi de 1000</li> 
    <li>Non, car rien ne garantit que le programme s'exécutera de la même façon sur une autre machine.</li>
</ul>

### 5. SYNTHESE

> **Activité 7a.** <br>
> On considère un petit système embarqué : un **microcontrôleur** relié à trois **LED A, B, C**. Une LED peut être éteinte ou éclairée et on peut configurer sa couleur. On dispose de trois programmes qui affichent des signaux lumineux en faisant clignoter les LED. Chaque programme possède une LED primaire et une LED secondaire. <br>
> - Le programme P1 affiche ses signaux sur A (primaire) et B (secondaire) en vert.<br>
> - Le programme P2 affiche ses signaux sur B (primaire) et C (secondaire) en orange.<br>
> - Le programme P3 affiche ses signaux sur C (primaire) et A (secondaire) en rouge.<br>
>
> Comme les LED ne peuvent pas être configurées dans deux couleurs en même temps, le système propose deux primitives **acquerirLED(nom)** et **rendreLED(nom)** qui permettent respectivement d'acquérir et de relâcher une LED. Si une LED est déjà acquise par un prograppe Pi alors acquerirLED() dans un programme Pj bloque Pj (i différent de j).<br>
> On suppose que chacun des trois programmes P1, P2, P3 effectue les actions suivantes en boucle :<br>
> 1. acquérir la LED primaire
> 2. acquérir la LED secondaire
> 3. configurer les couleurs
> 4. émettre des signaux
> 5. rendre la LED secondaire
> 6. rendre la LED primaire
> recommencer en 1
>
> **Montrer** qu'il existe un entrelacement des exécutions qui place **P1, P2 et P3 en interblocage**.

<p style="color:blue; font-weight:bold">CORRECTION 7a</p><br>
<span style="color:blue;"> On considère l'enchaînement suivant :</span></br>
<ul style="color:blue;">
    <li>P1 acquiert A (étape 1) puis est interrompu.</li>
    <li>P2 acquiert B (étape 1) puis est interrompu.</li>
    <li>P3 acquiert C (étape 1) puis est interrompu.</li>
    <li>P1 tente d'acquérir B, sa LED secondaire (étape 2). Il est bloqué car B est tenue par P2</li>
    <li>P2 tente d'acquérir C, (étape 2). Il est bloqué car C est tenue par P3</li>
    <li>P2 tente d'acquérir A, (étape 2). Il est bloqué car A est tenue par P1</li>
</ul>
<span style="color:blue;">
On obtient la boucle A -> P1 -> B -> P2 -> C -> P3 -> A <br>
A ce stade, les trois processus sont bloqués dans une attente circulaire d'une ressource tenue par un autre processus. Ils sont en <strong>interblocage</strong>. 
</span>

In [ ]:
# Correction 7b
# MICROLED - Illustration de l'interblocage dans la synthèse
# A télécharger à partir de https://gist.github.com/WebGE/f24c17bb13f10b38eaf21725451e3754
import threading

VERROU_LED={}
VERROU_LED['A']=threading.Lock()
VERROU_LED['B']=threading.Lock()
VERROU_LED['C']=threading.Lock()

def acquerirLED(led):
    VERROU_LED[led].acquire()

def rendreLED(led):
    VERROU_LED[led].release()
    
def prog(numproc,ledprim,ledsec):
    while True:
        acquerirLED(ledprim)
        print(f"Acquisition de {ledprim} par le programme {numproc}")
        acquerirLED(ledsec)
        print(f"Acquisition de {ledsec} par le programme {numproc}")
        print(f"Configuration de {ledprim} {ledsec} et envoi du signal")
        rendreLED(ledsec)
        rendreLED(ledprim)

p1 = threading.Thread(target=prog, args=[1,'A','B'])
p2 = threading.Thread(target=prog, args=[2,'B','C'])
p3 = threading.Thread(target=prog, args=[3,'C','A'])

p1.start()
p2.start()
p3.start()

### 6. RESUME
Les **threads** ou **processus légers** sont des "sous-processus" s'exécutant de manière concurente. L'accès à des ressources par plusieurs threads peut être protégé par des **verrous**. Une portion de code comprise entre l'acquisition et le relâchement d'un verrou s'appelle une **section critique**. 